In [11]:
import numpy as np
import pandas as pd
import math
import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize
from operator import itemgetter
import nltk
from nltk.stem import WordNetLemmatizer
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
       return round(dot(A, B)/(norm(A)*norm(B)),3)



with open('top_songs.pickle', 'rb') as handle:
    dic = pickle.load(handle)

# data = pd.read_csv('mylyrics00.csv')

singer_list=list(dic.keys())
song_list=list(dic.values())



stop_word = set(stopwords.words('english'))
doc = []
for i in range(len(song_list)):
    tmp=""
    for key in song_list[i].keys():
        temp=song_list[i][key]
        tmp=tmp+" "+temp

    
    
    list_temp = regexp_tokenize(tmp.lower(), "[a-z]['a-z]*")

    result = []
    for w in list_temp:
        if w not in stop_word:
            result.append(w)
    doc.append([singer_list[i],result])
singer=singer_list
        
# singer = (list(set(list(data.iloc[:-1]['artist']))))
# print(singer)



# for i in range(len(singer)):
#     sing = singer[i]
#     temp = ""
#     #print(len(data.iloc[:-1]['lyrics']))
#     for j in range(len(data.iloc[:-1]['lyrics'])):
#         if sing == data.iloc[j]['artist']:
#             #a = data.iloc[j]['lyrics'].split(' ')
#             temp = temp + " " + data.iloc[j]['lyrics']
#     list_temp = regexp_tokenize(temp.lower(), "[a-z]['a-z]*")
    
#     result = []
#     for w in list_temp:
#         if w not in stop_word:
#             result.append(w)
    
    
#     doc.append([singer[i],result])

In [12]:
class List:
    class Node:
        def __init__(self, doc,dtf): #node는 doc_number,dtf link로 구성
            self.doc = doc
            self.dtf=dtf
            self.next = None

    def __init__(self,term): #head는 term,freq,link로 구성
        self.head = None
        self.term=term
        self.freq = 0

    def freq(self):
        return self.freq

    def term(self):
        return self.term

    def add(self,doc,dtf):
        p=self.head
        if p==None:
            self.head=self.Node(doc,dtf)
        else:
            while (p.next != None):
                p=p.next
            p.next = self.Node(doc,dtf)
        self.freq += 1

    def print_list(self): # doc_number를 출력하기 위한 함수
        p=self.head
        res=[]
        while p is not None:
            res.append(p.doc)
            p=p.next
        return res

In [13]:
def listing(index): #LINKED LIST head:term,doc_freq // node: doc_num, freq in doc_num
    list_set=[]

    tmp_t=index[0][0]
    tmp_d=index[0][1]
    cnt=1
    for i in range(1,len(index)):
        tmp_term=index[i][0]
        tmp_doc=index[i][1]

        if(tmp_term==tmp_t and tmp_doc==tmp_d):
           cnt+=1
        else:
            list_set.append([tmp_t,tmp_d,cnt])
            cnt=1
        tmp_t=tmp_term
        tmp_d=tmp_doc
    list_set.append([tmp_t, tmp_d, cnt])

    list_all=[]
    term=list_set[0][0]
    list_t=List(term)
    list_t.add(list_set[0][1],list_set[0][2])
    for i in range(1,len(list_set)):
        if(term!=list_set[i][0]):
            list_all.append(list_t)
            list_t=List(list_set[i][0])
        list_t.add(list_set[i][1],list_set[i][2])
        term=list_set[i][0]
    list_all.append(list_t)

    return list_all

def indexing():  #vsm
    vsm_word1 = []
    index_doc=[]
    indexed_list1=[]
    for i in range(len(doc)):
        for j in range(len(doc[i][1])):
            index_doc.append([doc[i][1][j],i])

    index_doc.sort(key=itemgetter(0))

    indexed_list1=listing(index_doc)

    for i in range(len(indexed_list1)):  # 각 단어별 weight 계산 단어 1개
        vsm_word1.append([0 for j in range(len(doc) + 1)])
        vsm_word1[i][0] = indexed_list1[i].term
        p = indexed_list1[i].head
        while (p != None):
            w = (1 + math.log2(p.dtf)) * math.log2(len(doc) / indexed_list1[i].freq)
            #w=p.dtf
            vsm_word1[i][p.doc + 1] = float(w)
            p = p.next
    return vsm_word1

vsm=indexing()


In [20]:

vsm_=np.zeros((len(vsm),len(vsm[0])-1))
for i in range(len(vsm_)): #vsm
        for j in range(len(vsm_[i])):
            vsm_[i][j]=vsm[i][j+1]
vsm_t=vsm_.T

In [21]:
cos_sim_a=[]

for i in range(len(singer)):
    list_sim=[]
    for j in range(len(singer)):
        list_sim.append(cos_sim(vsm_t[i],vsm_t[j]))
  
    cos_sim_a.append(list_sim)


# Max accuracy

In [22]:
for i in range(len(cos_sim_a)):
    test = cos_sim_a[i][:]
    test.pop(test.index(max(test)))
    a = cos_sim_a[i].index(max(test))
    print("" + singer[i] +"      "  + singer[a]+ " :   " + str(max(test)))

eminem      dr-dre :   0.161
ghostface-killah      50-cent :   0.163
50-cent      ghostface-killah :   0.163
dr-dre      2pac :   0.184
drake      chris-brown :   0.118
2pac      dr-dre :   0.184
beyonce-knowles      chris-brown :   0.119
ariana-grande      eminem :   0.068
chris-brown      eminem :   0.136
glen-campbell      eddy-arnold :   0.109
ciara      chris-brown :   0.122
ed-sheeran      eminem :   0.1
garth-brooks      glen-campbell :   0.081
billy-ray-cyrus      bryan-adams :   0.078
brad-paisley      eminem :   0.082
bryan-adams      the-calling :   0.084
eddy-arnold      glen-campbell :   0.109
beatles      eminem :   0.072
coldplay      the-calling :   0.053
the-calling      bryan-adams :   0.084
the-doors      ghostface-killah :   0.068


# Min accuracy

In [23]:
for i in range(len(cos_sim_a)):
    a = cos_sim_a[i].index(min(cos_sim_a[i]))
    print("" + singer[i] +"/"  + singer[a]+ " :   " + str(min(cos_sim_a[i])))

eminem/coldplay :   0.05
ghostface-killah/the-calling :   0.045
50-cent/coldplay :   0.038
dr-dre/the-calling :   0.037
drake/coldplay :   0.032
2pac/coldplay :   0.042
beyonce-knowles/eddy-arnold :   0.033
ariana-grande/coldplay :   0.029
chris-brown/glen-campbell :   0.038
glen-campbell/ciara :   0.028
ciara/the-calling :   0.024
ed-sheeran/ciara :   0.042
garth-brooks/ciara :   0.037
billy-ray-cyrus/ciara :   0.032
brad-paisley/ariana-grande :   0.035
bryan-adams/ciara :   0.043
eddy-arnold/ciara :   0.026
beatles/coldplay :   0.035
coldplay/ciara :   0.025
the-calling/ciara :   0.024
the-doors/ciara :   0.031
